In [1]:
def read_PAM250_table(path):
    """
    Reads BLOSUM table
    the key = row_alpha + col_alpha
    """
    fr = open(path,"r")
    res_mat = {}
    alpha = fr.readline().strip().split()
    for line in fr.readlines():
        row_alpha = line[0]
        for col_alpha, val in zip(alpha,line.split()[1:]):
            key = row_alpha.strip() + col_alpha.strip()
            res_mat[key] = int(val)
    return res_mat

In [1]:
def LocalAllignmentGraph(seq1, seq2, table, indel_penalty):
    """
    Creates 
    seq1 -> represents rowumn
    seq2 -> represents col
    Returns (path_graph, max_score)
    """
    # need to initialize with indel penalty
    match_graph = [ [0 for row in range(len(seq2)+1)] for col in range(len(seq1)+1)]
    for row in range(0, len(seq1)+1):
        match_graph[row][0] = -indel_penalty*row  
    for col in range(0, len(seq2)+1):
        match_graph[0][col] = -indel_penalty*col
    
    max_score = 0
    fr_row = None
    fr_col = None
    
    bt_graph = [ ["-" for row in range(len(seq2)+1)] for col in range(len(seq1)+1)]
    for row in range(1, len(seq1)+1):
        for col in range(1, len(seq2)+1):
            key = seq1[row-1] + seq2[col-1]
            match_score = table[key]
            
            match_graph[row][col] = max(0,match_graph[row-1][col] - indel_penalty, match_graph[row][col-1] - indel_penalty, match_graph[row-1][col-1]+match_score)
            if match_graph[row][col] == match_graph[row-1][col] - indel_penalty:
                bt_graph[row][col] = "D"
            elif match_graph[row][col] == match_graph[row][col-1] - indel_penalty:
                bt_graph[row][col] = "R"
            elif match_graph[row][col] == match_graph[row-1][col-1]+match_score:
                bt_graph[row][col] = "DG"
            elif match_graph[row][col] == 0:
                bt_graph[row][col] = "FR"
            
            # Check if it pays off to have a free ride to the sink
            if match_graph[row][col] > max_score:
                max_score = match_graph[row][col]
                fr_row = row
                fr_col = col
    
    # Mark free ride to source
#     bt_graph[fr_row][fr_col] = "FR_TO_SINK"
    match_graph[fr_row][fr_col] = max_score
#     print(match_graph)
#     print(bt_graph)
    
    return bt_graph, max_score, fr_row, fr_col

In [43]:
def ConstructLocalAllignment(backtrack,seq1,seq2,start_row, start_col):
    """
    Constructs globall allignment from the allignment graph
    """
    ALL_1=""
    ALL_2=""
    i=start_row
    j=start_col
    while i > 0 and j > 0:
        if backtrack[i][j] == "DG":
            ALL_1+=seq1[i-1]
            ALL_2+=seq2[j-1]
            i = i-1
            j = j-1
        # Down
        elif backtrack[i][j] == "D":
            ALL_1+=seq1[i-1]
            ALL_2+="-"
            i = i-1
        # Right
        elif backtrack[i][j] == "R":
            ALL_1+="-"
            ALL_2+=seq2[j-1]
            j = j-1
        # Free Ride
        elif backtrack[i][j] == "FR":
            # we have a Free ride to source, so we end
            break
    
    print(ALL_1[::-1])
    print(ALL_2[::-1])

In [44]:
def read_seq_file(path):
    with open(path) as f:
        seq1 = f.readline().strip()
        seq2 = f.readline().strip()
    return seq1, seq2

In [45]:
def LocalAllignmentProblem(seq1, seq2, penalty, table):
    all_graph, match_g,fr_r, fr_c = LocalAllignmentGraph(seq1,seq2,table,penalty)
    print(match_g)
    ConstructLocalAllignment(all_graph,seq1, seq2, fr_r, fr_c)

In [46]:
pam250 = read_PAM250_table("/Users/marcisin/Desktop/PAM250.txt")

In [47]:
s1="MEANLY"
s2="PENALTY"
pam250 = read_PAM250_table("/Users/marcisin/Desktop/PAM250.txt")
penalty=5
LocalAllignmentProblem(s1,s2,penalty,pam250)

[[0, -5, -10, -15, -20, -25, -30, -35], [-5, 0, 0, 0, 0, 0, 0, 0], [-10, 0, 4, 1, 0, 0, 0, 0], [-15, 0, 0, 4, 3, 0, 1, 0], [-20, 0, 1, 2, 4, 0, 0, 0], [-25, 0, 0, 0, 0, 10, 5, 0], [-30, 0, 0, 0, 0, 5, 7, 15]]
[['-', '-', '-', '-', '-', '-', '-', '-'], ['-', 'FR', 'FR', 'FR', 'FR', 'FR', 'FR', 'FR'], ['-', 'FR', 'DG', 'DG', 'DG', 'FR', 'DG', 'FR'], ['-', 'FR', 'DG', 'DG', 'DG', 'FR', 'DG', 'FR'], ['-', 'FR', 'DG', 'DG', 'DG', 'DG', 'DG', 'FR'], ['-', 'FR', 'FR', 'FR', 'DG', 'DG', 'R', 'R'], ['-', 'FR', 'FR', 'FR', 'FR', 'D', 'DG', 'DG']]
15
EANL-Y
ENALTY


In [56]:
LocalAllignmentProblem('AAAAAASSSSSSSVVVVVVVVTTTTTTTLLLLLL','SSSSLLLLLTTTTTTTWWWWWWWWWWWPPPPPPPPPP',penalty,pam250)

33
TTTLLLLL
SSSLLLLL


In [57]:
path="/Users/marcisin/Downloads/dataset_247_10.txt"
s1, s2 =read_seq_file(path)
penalty=5
pam250 = read_PAM250_table("/Users/marcisin/Desktop/PAM250.txt")
LocalAllignmentProblem(s1,s2,penalty,pam250)

1189
QNRKTLFPDR-YS-FANGWSLYNLDMFINYVQPIRHRNIQYCITWNHLIQDASDF-HQF--MHQL-ARAHDCMMTHYARADT-VWCYCNAMNKLI-QSMRCGDI-D-RYFNGCYGPETVVAVQVWGNA-PQPGF-FMSMKYICKH-D--FF-YQN-CRS--CIQSKTIAQGVIIPM--EDDIV-RTYKE-W--W-S-FG--PK-DDFNKMCFVPKIIKYCEYWIQYAQ--FNISMCGRDKIHKWRKNCYHMQQE-IGF-CFAMM-E-FDNMMTFEFCPSQQLKG-GWIVKA-WH-C---MWQT--C--WQHGGLMFHMLSKCYYGIRPLRNDSEHTCQVTFWPY-D-MFAYNGKINF-HVMVGGGCEWACYAQ-P-M-----GMVGPPQMNDTHIENKVSL-S-----EP----M--LARHDYRMEHWAGDVSAYIGMAKVETRDDNMFLFMNHAIWAQP--LA-DCDKTG---Y----KEQTHVSEMKFPRQDFYQEMGCATLFGWKEGNQVRSVPGGCNPWCMKYHKTDWYVIKQVHYDMEKWDKNEQYIQMIPGRNSNDNW-LFCVMWNSHFKKAHRWGHTPADCEVWGTAYIRRQRHQF-QA--SCGWPNRYCHKIWSVRYKYPCGCLKYLKVMSKHVFVQKIYILEYSEME-CDRLEKEF-ERFCVYEYQCSTTL-NLP--AFCSCHI-M--FAL-IMM-SGSSREY----FYMRHRVDASV-SHLRDDGPPYIVYHKSR-DSGTYHKWTQNEAVSQQLCCTIKQ-AE-AH--M-WTETVLRMDYPMTWNDTARI-Q--V-M-WMTTTHHKEAPNQDW--ETRNMKAPFVWRKFQGNWCSGSHPHTRCYLIEMTDNPVNHWE-EC
HIHITL-AGWAFPWLTAVWPRFD-CMFDPY-EKFWQQTMQLHAQQNESKHRCYIYVPQFDAIHYIQGTSWQ-IGNHKGIAESHTWHY-WRMEAEICDATQQTTLNDIRLFET-FD-DWASPQGS